In [65]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta

# === PARAMETERS ===
ticker = "^NSEI"
portfolio_value = float(input("Enter your portfolio value in ₹: "))  # Ask for corpus
lot_size = 50
beta_map = {'Bullish': 1.5, 'Neutral': 0.75, 'Bearish': 0.5}

# === FETCH DAILY DATA (need much more data for proper 200DMA + historical months) ===
end_date = datetime.now()
start_date = end_date - timedelta(days=800)  # Get 800 days to ensure we have enough historical months

data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), 
                   end=end_date.strftime('%Y-%m-%d'), interval="1d")

if data.empty:
    raise ValueError("No data fetched. Check ticker or internet connection.")

# Determine price column
price_col = 'Close' if 'Close' in data.columns else 'Adj Close'
data = data[[price_col]].copy()
data.rename(columns={price_col: 'Price'}, inplace=True)
data = data.dropna()

# === CHECK IF TODAY IS MONTH-END ===
today = data.index[-1].date()  # last available trading day
last_day_of_month = (today + MonthEnd(0)).date()
if today != last_day_of_month:
    print(f"⚠️ Warning: Today ({today}) is not month-end ({last_day_of_month}).")
    print("Calculation will still run using the latest available price.\n")

# === CALCULATE 200DMA AND 3-MONTH SLOPE PROPERLY ===
# Calculate 200DMA with proper minimum periods
data['200DMA'] = data['Price'].rolling(window=200, min_periods=200).mean()

# Calculate 3-month slope correctly (63 trading days ago)
data['200DMA_3m_ago'] = data['200DMA'].shift(63)
data['DMA_slope'] = data['200DMA'] - data['200DMA_3m_ago']

# Get only valid data (where we have both 200DMA and slope)
valid_data = data.dropna()

if valid_data.empty:
    raise ValueError("Not enough data for 200DMA calculation. Need at least 263+ trading days.")

latest = valid_data.iloc[-1]
spot = latest['Price'].item()
dma_200 = latest['200DMA'].item()
slope = latest['DMA_slope'].item()
as_of = latest.name.date()

# === DETERMINE TREND ===
if spot > dma_200 and slope > 0:
    trend = 'Bullish'
elif spot > dma_200:
    trend = 'Neutral'
else:
    trend = 'Bearish'

beta = beta_map[trend]

# === FUTURES TARGET ===
futures_notional = (beta - 1) * portfolio_value
lots = int(round(futures_notional / (spot * lot_size)))

# === DISPLAY TABLE ===
summary = pd.DataFrame({
    "Date": [as_of],
    "Nifty Spot": [f"₹{spot:,.0f}"],
    "200DMA": [f"₹{dma_200:,.0f}"],
    "Trend": [trend],
    "Beta (Overlay)": [f"{beta}x"],
    "Portfolio (₹ Cr)": [f"₹{portfolio_value/1e7:.1f} Cr"],
    "Futures Notional": [f"₹{futures_notional:,.0f}"],
    "Lots to Hold": [abs(lots)],
    "Position": ["🟢 LONG" if lots > 0 else "🔴 SHORT" if lots < 0 else "⚪ FLAT"]
})

print("\n" + "🎯" * 27)
print("🎯 CURRENT MONTH - ACTIONABLE TRADING SIGNAL 🎯")
print("🎯" * 27)
print("⚠️  ONLY THIS SIGNAL IS FOR ACTUAL TRADING")
print("="*80)

# Enhanced current month display
current_summary = pd.DataFrame({
    "📅 Date": [as_of.strftime('%d-%b-%Y')],
    "📈 Nifty": [f"₹{spot:,.0f}"],
    "📊 200DMA": [f"₹{dma_200:,.0f}"],
    "🎭 Signal": [f"🔥 {trend}" if trend == 'Bullish' else f"⚡ {trend}" if trend == 'Neutral' else f"❄️ {trend}"],
    "⚖️ Beta": [f"🚀 {beta}x" if beta > 1 else f"🛡️ {beta}x" if beta < 1 else f"➡️ {beta}x"],
    "💰 Portfolio": [f"₹{portfolio_value/1e7:.1f} Cr"],
    "📋 Lots": [f"⭐ {abs(lots)}" if lots != 0 else "⚪ 0"],
    "🎬 ACTION": [f"🟢 BUY {abs(lots)} LOTS" if lots > 0 else f"🔴 SELL {abs(lots)} LOTS" if lots < 0 else "⚪ HOLD FLAT"]
})

print(current_summary.to_string(index=False))
print("="*80)

# Action highlight box
print("\n" + "┌" + "─" * 78 + "┐")
if lots > 0:
    print(f"│ 🎯 EXECUTE: BUY {abs(lots)} NIFTY FUTURES LOTS (LONG POSITION)              │")
    print(f"│ 💡 Reason: {trend} market trend detected                                │")
elif lots < 0:
    print(f"│ 🎯 EXECUTE: SELL {abs(lots)} NIFTY FUTURES LOTS (SHORT POSITION)             │")
    print(f"│ 💡 Reason: {trend} market trend detected                                │")
else:
    print("│ 🎯 EXECUTE: NO FUTURES POSITION (STAY FLAT)                             │")
    print(f"│ 💡 Reason: {trend} market conditions                                    │")
print("└" + "─" * 78 + "┘")

# === LAST 3 MONTHS STRATEGY DECISIONS ===
monthly_data = valid_data.resample('M').last()  # Get month-end data
last_3_months = monthly_data.tail(3)

print(f"\nDEBUG: Found {len(last_3_months)} months of data")  # Debug line

if len(last_3_months) >= 1:  # Changed from 3 to 1 to ensure it shows
    print("\n" + "="*80) 
    print("📈 LAST 3 MONTHS - SIMULATION ONLY (What Strategy Would Have Done)")
    print("⚠️  WARNING: This is historical simulation - NOT actual trading advice")
    print("="*80)
    
    monthly_summary = []
    
    for date, row in last_3_months.iterrows():
        month_spot = row['Price'].item()  # Extract scalar value
        month_200dma = row['200DMA'].item()  # Extract scalar value
        month_slope = row['DMA_slope'].item()  # Extract scalar value
        
        # Determine trend for that month
        if month_spot > month_200dma and month_slope > 0:
            month_trend = 'Bullish'
        elif month_spot > month_200dma:
            month_trend = 'Neutral'
        else:
            month_trend = 'Bearish'
        
        month_beta = beta_map[month_trend]
        
        # Calculate position for ₹1 Cr portfolio (standard reference)
        reference_portfolio = 1e7  # ₹1 Cr reference
        month_futures_notional = (month_beta - 1) * reference_portfolio
        month_lots = int(round(month_futures_notional / (month_spot * lot_size)))
        
        monthly_summary.append({
            "Month": date.strftime('%b-%Y'),
            "Nifty Close": f"₹{month_spot:,.0f}",
            "200DMA": f"₹{month_200dma:,.0f}",
            "Signal": month_trend,
            "Beta": f"{month_beta}x",
            "Position": "🟢 LONG" if month_lots > 0 else "🔴 SHORT" if month_lots < 0 else "⚪ FLAT",
            "Lots": abs(month_lots) if month_lots != 0 else 0,
            "Action": f"{'Buy' if month_lots > 0 else 'Sell' if month_lots < 0 else 'Hold'} {abs(month_lots) if month_lots != 0 else 'FLAT'}"
        })
    
    monthly_df = pd.DataFrame(monthly_summary)
    print(monthly_df.to_string(index=False))
else:
    print("\n❌ Not enough monthly data available")
    
print("\n" + "="*80)

Enter your portfolio value in ₹: 10000000


/var/folders/59/27lpl20d01l3sj6sf_qj_1300000gn/T/ipykernel_72743/138685426.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'),
[*********************100%***********************]  1 of 1 completed

⚠️ Warning: Today (2025-08-26) is not month-end (2025-08-31).
Calculation will still run using the latest available price.


🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
🎯 CURRENT MONTH - ACTIONABLE TRADING SIGNAL 🎯
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
⚠️  ONLY THIS SIGNAL IS FOR ACTUAL TRADING
     📅 Date 📈 Nifty 📊 200DMA  🎭 Signal ⚖️ Beta 💰 Portfolio 📋 Lots     🎬 ACTION
26-Aug-2025 ₹24,712  ₹24,070 🔥 Bullish  🚀 1.5x     ₹1.0 Cr    ⭐ 4 🟢 BUY 4 LOTS

┌──────────────────────────────────────────────────────────────────────────────┐
│ 🎯 EXECUTE: BUY 4 NIFTY FUTURES LOTS (LONG POSITION)              │
│ 💡 Reason: Bullish market trend detected                                │
└──────────────────────────────────────────────────────────────────────────────┘

DEBUG: Found 3 months of data

📈 LAST 3 MONTHS - SIMULATION ONLY (What Strategy Would Have Done)
⚠️  WARNING: This is historical simulation - NOT actual trading advice
   Month Nifty Close  200DMA  Signal Beta Position  Lots Action
Jun-2025     ₹25,517 ₹24,084 Bul